In [1]:
## BEGIN OF TARGET CONFIGURATION

PORT = 16384
IPADDR = '10.3.0.255'
DESTINATIONS = ['10.2.0.14', '10.2.0.16']
REPEAT_WAIT_SEC = 20
REPETITION = 10

### END OF TARGET_CONFIGURATION

import sys
import os
import pathlib
import math
import argparse
import datetime
import numpy as np

from e7awgsw import CaptureModule, AWG, AwgCtrl, CaptureCtrl, WaveSequence, CaptureParam, plot_graph
from e7awgsw.labrad import RemoteAwgCtrl, RemoteCaptureCtrl


d = {
    'PATH_TO_QUBECALIB': '{}/qube-calib/qubecalib'.format(os.getcwd()),
    'PATH_TO_E7AWGSW': '{}/qube-calib/e7awg_sw'.format(os.getcwd()),
    'PATH_TO_QUBELSI': '{}/qube-calib/adi_api_mod/python/qubelsi'.format(os.getcwd()),
    'PATH_TO_QUBEMASTER': '{}/qube_master/software'.format(os.getcwd()),
}
for k, p in d.items():
    if not p in sys.path:
        sys.path.insert(1, p)

import qubecalib as qc
import qubecalib.meas
import qubemasterclient
import sequencerclient

repeat_wait_sec = REPEAT_WAIT_SEC * 125000000 # 125Mcycls = 1sec

client = qubemasterclient.QuBEMasterClient(IPADDR, PORT)
print('clear')
r, a = client.clear_clock(value=0)
print('kick')
r, a = client.kick_clock_synch(DESTINATIONS)

print('read')
cur_time = client.read_clock(value=0)
print('read result', cur_time, 'cycles')

for i in range(REPETITION):
    cur_time += repeat_wait_sec
    print('run @', cur_time)
    for a in DESTINATIONS:
        client = sequencerclient.SequencerClient(a, PORT)
        r, a = client.add_sequencer(cur_time) # make registration


open: 10.3.0.255:16384
clear
b'4\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
kick
kick: 0x0a02000e:16385
kick: 0x0a020010:16385
b'2\x00\x00\x00\x00\x00\x00\x00\n\x02\x00\x0e\x00\x00@\x01\n\x02\x00\x10\x00\x00@\x01'
read
b'0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
(b'3\x00\x00\x00\x00\x00\x00\x00X\xa4\x00\x00\x00\x00\x00\x00', ('10.3.0.255', 16384))
read result 42072 cycles
run @ 2500042072
open: 10.2.0.14:16384
open: 10.2.0.16:16384
run @ 5000042072
open: 10.2.0.14:16384
open: 10.2.0.16:16384
run @ 7500042072
open: 10.2.0.14:16384
open: 10.2.0.16:16384
run @ 10000042072
open: 10.2.0.14:16384
open: 10.2.0.16:16384
run @ 12500042072
open: 10.2.0.14:16384
open: 10.2.0.16:16384
run @ 15000042072
open: 10.2.0.14:16384
open: 10.2.0.16:16384
run @ 17500042072
open: 10.2.0.14:16384
open: 10.2.0.16:16384
run @ 20000042072
open: 10.2.0.14:16384
open: 10.2.0.16:16384
run @ 22500042072
open: 10.2.0.14:16384
open: 10.2.0.16:16384
run @ 25000042072
open: 10.2.0.